In [1]:
## Imports

# utility modules
import glob
import os
import sys
import re
import rasterio as rio


# the usual suspects:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import cartopy.crs as ccrs

# specialty modules
import h5py
import pyproj
import dask

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
cd

/home/jovyan


In [3]:
cd Land_Ice_Applications/readers

/home/jovyan/Land_Ice_Applications/readers


In [4]:
from read_HDF5_ATL03 import read_HDF5_ATL03
from get_ATL03_x_atc import get_ATL03_x_atc

### 1.2.2 Data used
Data for this tutorial are stored in a shared drive, accessible to all tutorial participants.  If you're getting data for yourself, you'll need to put it in a consistent place, and change this cell to match your directory.

In [5]:
data_root='shared/FirnAndMelt/ATL03/Antarctica/'

In [6]:
import cartopy.crs as ccrs
from rasterio import plot

# 2. Data in along-track format

# 2.1 ATL03 elevations

Before we start looking at the ATL06 data we've downloaded, let's have a look at some of the ATL03 data that were used to make them.  One of the source ATL03 files is in the shared folder, and we'll read it with Tyler Sutterley's excellent "read_HDF5_ATL03" function.

In [7]:
cd 

/home/jovyan


In [8]:
cd 'shared/FirnAndMelt/ATL03/Antarctica/'

/srv/shared/FirnAndMelt/ATL03/Antarctica


In [9]:
# read the data:
#rgt="0027"
#cycle="06"
# read the IS2 data with Tyler's ATL03 reader:
ATL03_file=  'processed_ATL03_20200203123745_05920612_003_01.h5'
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)

The data are returned in a set of dictionaries that mimic the structure of an ATL03 file.  To help visualize the data, we're going to calculate an along-track coordinate for every photon in the cloud (x_atc).  This is a slightly complex job, and there's a helper function in the readers directory that you can look at if you want the details.

In [10]:
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

In [11]:
IS2_atl03_mds['gt2r']

{'heights': {'delta_time': array([65968768.88493214, 65968768.88493214, 65968768.88493214, ...,
         65968780.56383242, 65968780.56383242, 65968780.56383242]),
  'dist_ph_across': array([-52.62856 , -52.638065, -52.626698, ..., -59.818478, -59.819874,
         -59.857765], dtype=float32),
  'dist_ph_along': array([ 0.48488307,  0.4367691 ,  0.49435243, ..., 19.510641  ,
         19.503958  , 19.32308   ], dtype=float32),
  'h_ph': array([197.02295, 179.9768 , 200.36516, ..., 104.63813, 102.27638,
          38.30188], dtype=float32),
  'lat_ph': array([-72.49992335, -72.49992377, -72.49992327, ..., -71.76736058,
         -71.76736064, -71.7673622 ]),
  'lon_ph': array([68.32057214, 68.32057261, 68.32057205, ..., 67.99997189,
         67.99997195, 67.99997371]),
  'pce_mframe_cnt': array([72830108, 72830108, 72830108, ..., 72830692, 72830692, 72830692],
        dtype=uint32),
  'ph_id_channel': array([90, 87, 26, ..., 24, 35, 28], dtype=uint8),
  'ph_id_count': array([1, 1, 1, ..., 2

In [12]:
df = pd.DataFrame(data={'height':IS2_atl03_mds['gt2r']['heights']['h_ph'],
                        'lat':  IS2_atl03_mds['gt2r']['heights']['lat_ph'],
                        'lon':  IS2_atl03_mds['gt2r']['heights']['lon_ph'],
                        'distance_along': IS2_atl03_mds['gt2r']['heights']['dist_ph_along'],
                        'signal_conf': IS2_atl03_mds['gt2r']['heights']['signal_conf_ph'][:,3] >= 2
                       },
                  index=IS2_atl03_mds['gt2r']['heights']['delta_time'])
df

,height,lat,lon,distance_along,signal_conf
6.596877e+07,197.022949,-72.499923,68.320572,0.484883,False
6.596877e+07,179.976807,-72.499924,68.320573,0.436769,False
6.596877e+07,200.365158,-72.499923,68.320572,0.494352,False
6.596877e+07,172.272217,-72.499924,68.320573,0.414983,False
6.596877e+07,167.983429,-72.499924,68.320573,0.402877,False
...,...,...,...,...,...
6.596878e+07,104.653831,-71.767361,67.999972,19.510685,True
6.596878e+07,102.633972,-71.767361,67.999972,19.504976,True
6.596878e+07,104.638130,-71.767361,67.999972,19.510641,True
6.596878e+07,102.276382,-71.767361,67.999972,19.503958,True


### 2.1.1 Plotting ATL03 photons
Now let's plot the ATL03 photons.  We'll plot all the photon heights as small black dots, then plot the photons that the ATL03 land-ice signal finder designates as surface (with low, medium, or high confidence) in green.  

In [13]:
#-- select the 2l beam from ATL03
D3 = IS2_atl03_mds['gt2r']

#-- create scatter plot of photon data (e.g., photon elevation vs x_atc)
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
#ax.plot(D3['heights']['lat_ph'], D3['heights']['h_ph'],'k.',markersize=0.25, label='all photons')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['lat_ph'][LMH][, D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
h_leg=ax.legend()

ax.set_xlabel('lat (deg)')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
dfg  = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']), crs='EPSG:4326')

In [15]:
dfg_sel = dfg[dfg['signal_conf']==True]

In [16]:
dfg_sel

,height,lat,lon,distance_along,signal_conf,geometry
6.596877e+07,121.134857,-72.499925,68.320574,0.270575,True,POINT (68.32057 -72.49993)
6.596877e+07,121.033951,-72.499925,68.320574,0.270293,True,POINT (68.32057 -72.49993)
6.596877e+07,121.355598,-72.499925,68.320574,0.271208,True,POINT (68.32057 -72.49993)
6.596877e+07,121.097557,-72.499925,68.320574,0.270468,True,POINT (68.32057 -72.49993)
6.596877e+07,121.336502,-72.499925,68.320574,0.271152,True,POINT (68.32057 -72.49993)
...,...,...,...,...,...,...
6.596878e+07,104.913750,-71.767361,67.999972,19.511423,True,POINT (67.99997 -71.76736)
6.596878e+07,104.653831,-71.767361,67.999972,19.510685,True,POINT (67.99997 -71.76736)
6.596878e+07,102.633972,-71.767361,67.999972,19.504976,True,POINT (67.99997 -71.76736)
6.596878e+07,104.638130,-71.767361,67.999972,19.510641,True,POINT (67.99997 -71.76736)


In [25]:
#bimodal_coeff =  (dfg_sel['height'].rolling(100).skew()**2+1)/dfg_sel['height'].rolling(100).kurt()
max_outlier = (dfg_sel['height'].rolling(100).max()-dfg_sel['height'].rolling(100).mean())/dfg_sel['height'].rolling(100).std()
min_outlier = (dfg_sel['height'].rolling(100).mean()-dfg_sel['height'].rolling(100).min())/dfg_sel['height'].rolling(100).std()

variance =  dfg_sel['height'].rolling(100).var()

fig, ax  = plt.subplots(nrows=3, figsize=(10,10))
ax[0].plot(dfg_sel['lat'].rolling(100).mean(),dfg_sel['height'].rolling(100).var())
ax[0].set_title('Variance')
ax[1].plot(dfg_sel['lat'].rolling(100).mean(),max_outlier)
ax[1].set_title('Grubb`s Test for Outliers')
ax[2].plot(dfg_sel['lat'],dfg_sel['height']);
ax[2].set_title('Photon Height (m)')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Photon Height (m)')

In [26]:
fig, ax = plt.subplots()

ax.scatter(min_outlier,max_outlier,s=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
from sklearn.cluster import KMeans
import sklearn.metrics 
from sklearn.metrics import pairwise_distances


In [18]:
def get_silhouette_score(X):
    Xdist = pairwise_distances(X)
    kmeans = KMeans(n_clusters=2).fit(Xdist)
    labels = kmeans.labels_
    return sklearn.metrics.silhouette_score(X,labels, metric='euclidean')
     



In [21]:
get_silhouette_score(X[50:1050])

0.5991076

In [49]:
subset  = dfg_sel[dfg_sel['lat']<(-72.3)]
subset  =  subset[subset['lat']>-72.4]

In [50]:
subset

,height,lat,lon,distance_along,signal_conf,geometry
6.596877e+07,136.861130,-72.399997,68.275495,6.252609,True,POINT (68.27550 -72.40000)
6.596877e+07,137.008133,-72.399997,68.275495,6.253027,True,POINT (68.27550 -72.40000)
6.596877e+07,137.028610,-72.399997,68.275495,6.253089,True,POINT (68.27549 -72.40000)
6.596877e+07,137.201370,-72.399997,68.275495,6.253581,True,POINT (68.27549 -72.40000)
6.596877e+07,136.931061,-72.399997,68.275495,6.252812,True,POINT (68.27550 -72.40000)
...,...,...,...,...,...,...
6.596877e+07,145.142181,-72.300002,68.230838,18.641714,True,POINT (68.23084 -72.30000)
6.596877e+07,145.717163,-72.300002,68.230838,18.643297,True,POINT (68.23084 -72.30000)
6.596877e+07,143.861649,-72.300002,68.230838,18.638069,True,POINT (68.23084 -72.30000)
6.596877e+07,145.289093,-72.300002,68.230838,18.642136,True,POINT (68.23084 -72.30000)


In [51]:
height_array = subset['height'].values
dist_array = subset['distance_along'].values


In [52]:
X = np.array((height_array, dist_array)).T


In [57]:
get_silhouette_score(X[0:1000])

0.61292464

In [80]:
silhouette = [0]
win_size = 500
for i in np.arange(0,int(len(height_array)-win_size/2),int(win_size/2)):
    window = X[i:i+win_size]
    score  = get_silhouette_score(window)
    silhouette  = np.append(silhouette, score)

In [83]:
fig, ax  = plt.subplots(nrows=2,  figsize=(5,10))
ax[0].plot(silhouette[1:])
ax[0].set_title('Silhouette Score of Cluster Tightness')
ax[1].plot(subset['height'])
ax[1].set_title('height (m)')

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'height (m)')